### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [2]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 2s 0us/step


In [5]:
(X_train.shape) , (X_test.shape) , (y_train.shape), (y_test.shape) 

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 10), (10000, 10))

In [6]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow(X_train, y_train, batch_size=9)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow(X_test, y_test, batch_size=9)

In [7]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
num_filter = 12
dropout_rate = 0
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
#First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Forth_Block = denseblock(Third_Transition, num_filter, dropout_rate)
Forth_Transition = transition(Forth_Block, num_filter, dropout_rate)

Fifth_Block = denseblock(Forth_Transition, num_filter, dropout_rate)
Fifth_Transition = transition(Fifth_Block, num_filter, dropout_rate)

#Last_Block = denseblock(Fifth_Transition,  num_filter, dropout_rate)
Last_Block = denseblock(Forth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [8]:
num_filter = 12
dropout_rate = 0
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(40, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D,20, dropout_rate)
First_Transition = transition(First_Block, 75, dropout_rate)

Second_Block = denseblock(First_Transition, 20, dropout_rate)
Second_Transition = transition(Second_Block, 40, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, 40, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [9]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 40)   1080        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 40)   160         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 40)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [15]:

from tensorflow.keras.callbacks import ModelCheckpoint
filepath="model_save/model.{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')



In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam', metrics=['accuracy'])

In [18]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.1,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow(X_train, y_train, batch_size=80)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)
steps_epoch = int(X_train.shape[0] / 40)


test_generator = test_datagen.flow(X_test, y_test, batch_size=9)
history = model.fit(train_generator ,epochs=290, validation_data=test_generator, verbose=2,callbacks=checkpoint)

Epoch 1/290
625/625 - 103s - loss: 1.6267 - accuracy: 0.4048 - val_loss: 1.4905 - val_accuracy: 0.4548

Epoch 00001: val_accuracy improved from -inf to 0.45480, saving model to model_save/model.01-0.45.h5
Epoch 2/290
625/625 - 90s - loss: 1.2927 - accuracy: 0.5349 - val_loss: 1.1202 - val_accuracy: 0.6043

Epoch 00002: val_accuracy improved from 0.45480 to 0.60430, saving model to model_save/model.02-0.60.h5
Epoch 3/290
625/625 - 90s - loss: 1.1375 - accuracy: 0.5915 - val_loss: 1.1536 - val_accuracy: 0.6073

Epoch 00003: val_accuracy improved from 0.60430 to 0.60730, saving model to model_save/model.03-0.61.h5
Epoch 4/290
625/625 - 90s - loss: 1.0449 - accuracy: 0.6300 - val_loss: 1.2896 - val_accuracy: 0.5766

Epoch 00004: val_accuracy did not improve from 0.60730
Epoch 5/290
625/625 - 90s - loss: 0.9683 - accuracy: 0.6579 - val_loss: 1.2816 - val_accuracy: 0.5879

Epoch 00005: val_accuracy did not improve from 0.60730
Epoch 6/290
625/625 - 90s - loss: 0.9013 - accuracy: 0.6816 - val

In [27]:
from keras.models import load_model
model_ = load_model('model.290-0.90.h5')
# Test the model
score = model_.evaluate(test_generator ,verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1112/1112 [==============================] - 9s 7ms/step - loss: 0.3515 - accuracy: 0.9006
Test loss: 0.3514561951160431
Test accuracy: 0.900600016117096
